<a href="https://colab.research.google.com/github/AliyaMZ/Python/blob/main/Task_OOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Вариант 8. Учет и управление рестораном**

Разработайте программное обеспечение для учета и управления рестораном. Программа должна предоставлять возможность создания записей о блюдах, ингредиентах, сотрудниках, заказах и других данных. Также программа должна поддерживать выполнение различных операций с данными, таких как добавление, редактирование, удаление, поиск и анализ.

In [25]:
from abc import ABC, abstractmethod, ABCMeta
from dataclasses import dataclass
from datetime import datetime
from enum import Enum
from typing import List, Dict, Optional, Type, TypeVar, Any
import logging


class InvalidItemError(Exception):
    """Исключение для некорректных данных пункта меню"""
    pass

class PermissionDeniedError(Exception):
    """Исключение для отсутствия прав доступа"""
    pass

class OrderNotFoundError(Exception):
    """Исключение для отсутствия заказа"""
    pass


class Preparable(ABC):
    @abstractmethod
    def prepare(self):
        """Метод для описания процесса приготовления"""
        pass

class Reportable(ABC):
    @abstractmethod
    def generate_report(self) -> str:
        """Метод для генерации отчетов"""
        pass


class LoggingMixin:
    def log_action(self, action: str):
        """Логирование действий"""
        logging.info(f"{datetime.now()}: {action}")
        print(f"LOG: {action}")

class NotificationMixin:
    def send_notification(self, message: str):
        """Отправка уведомлений"""
        print(f"NOTIFICATION: {message}")

class MenuItemMeta(ABCMeta):
    _registry: Dict[str, Type['MenuItem']] = {}

    def __new__(cls, name, bases, namespace):
        new_class = super().__new__(cls, name, bases, namespace)
        if name != 'MenuItem':
            cls._registry[name.lower()] = new_class
        return new_class

    @classmethod
    def get_item_class(cls, item_type: str) -> Type['MenuItem']:
        """Получение класса по типу пункта меню"""
        return cls._registry.get(item_type.lower())


class MenuItem(ABC, metaclass=MenuItemMeta):
    def __init__(self, item_id: int, name: str, price: float, category: str, is_available: bool = True):
        self._item_id = item_id
        self._name = name
        self._price = price
        self._category = category
        self._is_available = is_available

    @abstractmethod
    def calculate_cost(self) -> float:
        """Абстрактный метод для расчета стоимости"""
        pass


    @property
    def item_id(self) -> int:
        return self._item_id

    @item_id.setter
    def item_id(self, value: int):
        self._item_id = value

    @property
    def name(self) -> str:
        return self._name

    @name.setter
    def name(self, value: str):
        self._name = value

    @property
    def price(self) -> float:
        return self._price

    @price.setter
    def price(self, value: float):
        if value < 0:
            raise InvalidItemError("Price cannot be negative")
        self._price = value

    @property
    def category(self) -> str:
        return self._category

    @category.setter
    def category(self, value: str):
        self._category = value

    @property
    def is_available(self) -> bool:
        return self._is_available

    @is_available.setter
    def is_available(self, value: bool):
        self._is_available = value

    def __str__(self) -> str:
        return f"Пункт меню: {self._name}, Цена: {self._price}, Категория: {self._category}"

    def __lt__(self, other) -> bool:
        if isinstance(other, MenuItem):
            return self._price < other.price
        return NotImplemented

    def __gt__(self, other) -> bool:
        if isinstance(other, MenuItem):
            return self._price > other.price
        return NotImplemented


class Dish(MenuItem, Preparable, LoggingMixin, NotificationMixin):
    def __init__(self, item_id: int, name: str, price: float, category: str,
                 ingredients: List[str], is_available: bool = True):
        super().__init__(item_id, name, price, category, is_available)
        self._ingredients = ingredients

    @property
    def ingredients(self) -> List[str]:
        return self._ingredients

    @ingredients.setter
    def ingredients(self, value: List[str]):
        self._ingredients = value

    def calculate_cost(self) -> float:
        # Пример: скидка 10% на сезонные блюда
        discount = 0.1 if "сезонное" in self._category.lower() else 0
        return self._price * (1 - discount)

    def prepare(self):
        self.log_action(f"Начато приготовление блюда: {self._name}")
        print(f"Готовим {self._name} с ингредиентами: {', '.join(self._ingredients)}")
        self.send_notification(f"Блюдо {self._name} готовится")

    def __str__(self) -> str:
        return (f"Блюдо: {self._name}, Ингредиенты: {', '.join(self._ingredients)}, "
                f"Цена: {self._price}, Категория: {self._category}")

class Drink(MenuItem, Preparable, LoggingMixin, NotificationMixin):
    def __init__(self, item_id: int, name: str, price: float, category: str,
                 volume: float, is_available: bool = True):
        super().__init__(item_id, name, price, category, is_available)
        self._volume = volume

    @property
    def volume(self) -> float:
        return self._volume

    @volume.setter
    def volume(self, value: float):
        self._volume = value

    def calculate_cost(self) -> float:
        # Пример: скидка 5% на напитки объемом более 0.5 л
        discount = 0.05 if self._volume > 0.5 else 0
        return self._price * (1 - discount)

    def prepare(self):
        self.log_action(f"Начато приготовление напитка: {self._name}")
        print(f"Готовим {self._name} объемом {self._volume}л")
        self.send_notification(f"Напиток {self._name} готовится")

    def __str__(self) -> str:
        return f"Напиток: {self._name}, Объем: {self._volume}л, Цена: {self._price}"

class Dessert(MenuItem, Preparable, LoggingMixin, NotificationMixin):
    def __init__(self, item_id: int, name: str, price: float, category: str,
                 calories: int, is_available: bool = True):
        super().__init__(item_id, name, price, category, is_available)
        self._calories = calories

    @property
    def calories(self) -> int:
        return self._calories

    @calories.setter
    def calories(self, value: int):
        self._calories = value

    def calculate_cost(self) -> float:
        # Пример: скидка 15% на десерты с калорийностью менее 300
        discount = 0.15 if self._calories < 300 else 0
        return self._price * (1 - discount)

    def prepare(self):
        self.log_action(f"Начато приготовление десерта: {self._name}")
        print(f"Готовим {self._name} ({self._calories} ккал)")
        self.send_notification(f"Десерт {self._name} готовится")

    def __str__(self) -> str:
        return f"Десерт: {self._name}, Калорийность: {self._calories} ккал, Цена: {self._price}"


@dataclass
class Table:
    table_number: int
    capacity: int
    is_occupied: bool = False


class Order(LoggingMixin, NotificationMixin, Reportable):
    def __init__(self, order_id: int, customer: str, table: Table):
        self._order_id = order_id
        self._customer = customer
        self._table = table
        self._items: List[MenuItem] = []
        self._total_cost = 0.0
        self._created_at = datetime.now()
        self._status = "created"

    @property
    def order_id(self) -> int:
        return self._order_id

    @property
    def customer(self) -> str:
        return self._customer

    @property
    def table(self) -> Table:
        return self._table

    @property
    def items(self) -> List[MenuItem]:
        return self._items

    @property
    def total_cost(self) -> float:
        return self._total_cost

    @property
    def status(self) -> str:
        return self._status

    def add_item(self, item: MenuItem):
        if not item.is_available:
            raise InvalidItemError(f"Пункт меню {item.name} недоступен")
        self._items.append(item)
        self._calculate_total()
        self.log_action(f"Добавлен пункт меню {item.name} в заказ {self._order_id}")
        self.send_notification(f"Добавлен пункт меню в заказ {self._order_id}")

    def remove_item(self, item: MenuItem):
        if item in self._items:
            self._items.remove(item)
            self._calculate_total()
            self.log_action(f"Удален пункт меню {item.name} из заказа {self._order_id}")
        else:
            raise InvalidItemError(f"Пункт меню {item.name} не найден в заказе")

    def _calculate_total(self):
        self._total_cost = sum(item.calculate_cost() for item in self._items)

    def update_status(self, new_status: str):
        self._status = new_status
        self.log_action(f"Статус заказа {self._order_id} изменен на {new_status}")
        self.send_notification(f"Статус заказа {self._order_id}: {new_status}")

    def generate_report(self) -> str:
        items_info = "\n".join(str(item) for item in self._items)
        return (f"Отчет по заказу №{self._order_id}\n"
                f"Клиент: {self._customer}\n"
                f"Стол: {self._table.table_number}\n"
                f"Статус: {self._status}\n"
                f"Пункты меню:\n{items_info}\n"
                f"Общая стоимость: {self._total_cost:.2f}\n"
                f"Время создания: {self._created_at}")

    def __str__(self) -> str:
        return (f"Заказ №{self._order_id}, Клиент: {self._customer}, "
                f"Стол: {self._table.table_number}, Сумма: {self._total_cost:.2f}")


class MenuItemFactory:
    @staticmethod
    def create_item(item_type: str, **kwargs) -> MenuItem:
        item_class = MenuItemMeta.get_item_class(item_type)
        if not item_class:
            raise InvalidItemError(f"Неизвестный тип пункта меню: {item_type}")
        return item_class(**kwargs)


class OrderHandler(ABC):
    def __init__(self, successor: Optional['OrderHandler'] = None):
        self._successor = successor

    @abstractmethod
    def handle_request(self, order: Order, request: str):
        pass

class Waiter(OrderHandler):
    def handle_request(self, order: Order, request: str):
        if request == "изменить заказ":
            print(f"Официент обрабатывает запрос на изменение заказа {order.order_id}")
            order.update_status("изменен официантом")
        elif self._successor:
            self._successor.handle_request(order, request)

class Chef(OrderHandler):
    def handle_request(self, order: Order, request: str):
        if request == "приготовить заказ":
            print(f"Шеф-повар начинает готовить заказ {order.order_id}")
            for item in order.items:
                if isinstance(item, Preparable):
                    item.prepare()
            order.update_status("готовится")
        elif self._successor:
            self._successor.handle_request(order, request)

class Manager(OrderHandler):
    def handle_request(self, order: Order, request: str):
        if request == "отменить заказ":
            print(f"Менеджер отменяет заказ {order.order_id}")
            order.update_status("отменен")
        elif request == "подтвердить оплату":
            print(f"Менеджер подтверждает оплату заказа {order.order_id}")
            order.update_status("оплачен")
        elif self._successor:
            self._successor.handle_request(order, request)


class PreparationTemplate(ABC):
    def prepare_dish(self):
        self._gather_ingredients()
        self._cook()
        self._plate()
        self._serve()

    @abstractmethod
    def _gather_ingredients(self):
        pass

    @abstractmethod
    def _cook(self):
        pass

    @abstractmethod
    def _plate(self):
        pass

    @abstractmethod
    def _serve(self):
        pass

class PastaPreparation(PreparationTemplate):
    def _gather_ingredients(self):
        print("Собираем ингредиенты для пасты: макароны, соус, специи")

    def _cook(self):
        print("Варим макароны и готовим соус")

    def _plate(self):
        print("Выкладываем пасту на тарелку и поливаем соусом")

    def _serve(self):
        print("Подаем пасту с зеленью и сыром")


def check_permissions(required_role: str):
    def decorator(func):
        def wrapper(self, *args, **kwargs):
            if not hasattr(self, 'role'):
                raise PermissionDeniedError("Роль не определена")
            if self.role != required_role:
                raise PermissionDeniedError(f"Требуется роль {required_role}")
            return func(self, *args, **kwargs)
        return wrapper
    return decorator


class RestaurantManager:
    def __init__(self):
        self._menu_items: List[MenuItem] = []
        self._orders: List[Order] = []
        self._tables: List[Table] = []
        self._handler_chain = self._setup_handler_chain()

    def _setup_handler_chain(self) -> OrderHandler:
        manager = Manager()
        chef = Chef(manager)
        waiter = Waiter(chef)
        return waiter

    def add_menu_item(self, item: MenuItem):
        self._menu_items.append(item)

    def find_menu_item(self, name: str) -> Optional[MenuItem]:
        for item in self._menu_items:
            if item.name.lower() == name.lower():
                return item
        return None

    def create_order(self, customer: str, table_number: int) -> Order:
        table = next((t for t in self._tables if t.table_number == table_number), None)
        if not table:
            raise OrderNotFoundError(f"Стол {table_number} не найден")

        if table.is_occupied:
            raise OrderNotFoundError(f"Стол {table_number} уже занят")

        table.is_occupied = True
        order = Order(len(self._orders) + 1, customer, table)
        self._orders.append(order)
        return order

    def process_order_request(self, order_id: int, request: str):
        order = next((o for o in self._orders if o.order_id == order_id), None)
        if not order:
            raise OrderNotFoundError(f"Заказ {order_id} не найден")

        self._handler_chain.handle_request(order, request)

    def generate_order_report(self, order_id: int) -> str:
        order = next((o for o in self._orders if o.order_id == order_id), None)
        if not order:
            raise OrderNotFoundError(f"Заказ {order_id} не найден")

        return order.generate_report()

    def add_table(self, table_number: int, capacity: int):
        if any(t.table_number == table_number for t in self._tables):
            raise InvalidItemError(f"Стол {table_number} уже существует")
        self._tables.append(Table(table_number, capacity))


def main():

    logging.basicConfig(level=logging.INFO)

    manager = RestaurantManager()

    manager.add_table(1, 4)
    manager.add_table(2, 6)
    manager.add_table(3, 2)

    try:
        pasta = MenuItemFactory.create_item(
            "dish",
            item_id=1,
            name="Паста Карбонара",
            price=350,
            category="Основные блюда",
            ingredients=["макароны", "бекон", "яйца", "пармезан", "сливки"]
        )

        pizza = MenuItemFactory.create_item(
            "dish",
            item_id=2,
            name="Пицца Маргарита",
            price=450,
            category="Основные блюда",
            ingredients=["тесто", "томатный соус", "моцарелла", "базилик"]
        )

        cola = MenuItemFactory.create_item(
            "drink",
            item_id=3,
            name="Кола",
            price=100,
            category="Напитки",
            volume=0.5
        )

        tiramisu = MenuItemFactory.create_item(
            "dessert",
            item_id=4,
            name="Тирамису",
            price=300,
            category="Десерты",
            calories=450
        )

        manager.add_menu_item(pasta)
        manager.add_menu_item(pizza)
        manager.add_menu_item(cola)
        manager.add_menu_item(tiramisu)

        order = manager.create_order("Иван Иванов", 1)
        order.add_item(pasta)
        order.add_item(cola)
        order.add_item(tiramisu)

        print("\nИнформация о заказе:")
        print(order)

        print("\nОбработка заказа:")
        manager.process_order_request(order.order_id, "изменить заказ")
        manager.process_order_request(order.order_id, "приготовить заказ")
        manager.process_order_request(order.order_id, "подтвердить оплату")

        print("\nОтчет по заказу:")
        print(manager.generate_order_report(order.order_id))

        print("\nПриготовление пасты:")
        pasta_prep = PastaPreparation()
        pasta_prep.prepare_dish()

    except Exception as e:
        print(f"Ошибка: {e}")

if __name__ == "__main__":
    main()

LOG: Добавлен пункт меню Паста Карбонара в заказ 1
NOTIFICATION: Добавлен пункт меню в заказ 1
LOG: Добавлен пункт меню Кола в заказ 1
NOTIFICATION: Добавлен пункт меню в заказ 1
LOG: Добавлен пункт меню Тирамису в заказ 1
NOTIFICATION: Добавлен пункт меню в заказ 1

Информация о заказе:
Заказ №1, Клиент: Иван Иванов, Стол: 1, Сумма: 750.00

Обработка заказа:
Официент обрабатывает запрос на изменение заказа 1
LOG: Статус заказа 1 изменен на изменен официантом
NOTIFICATION: Статус заказа 1: изменен официантом
Шеф-повар начинает готовить заказ 1
LOG: Начато приготовление блюда: Паста Карбонара
Готовим Паста Карбонара с ингредиентами: макароны, бекон, яйца, пармезан, сливки
NOTIFICATION: Блюдо Паста Карбонара готовится
LOG: Начато приготовление напитка: Кола
Готовим Кола объемом 0.5л
NOTIFICATION: Напиток Кола готовится
LOG: Начато приготовление десерта: Тирамису
Готовим Тирамису (450 ккал)
NOTIFICATION: Десерт Тирамису готовится
LOG: Статус заказа 1 изменен на готовится
NOTIFICATION: Ста